In [7]:
from skimage.color import rgb2gray
import skimage.io as io
import numpy as np
import cv2 as cv2


In [9]:
def Get_lanes_bases(img):

    # Get the Histogram of the bottom half of the frame captured
    half_frame = img.shape[0] //2

    crop_half_frame = img[half_frame:,:]
    axis_Histogram = np.sum(crop_half_frame,axis = 0)

    # Get where is the max values located on the x-axis
    left_lane = np.argmax(axis_Histogram[:img.shape[1]//2])
    right_lane = (img.shape[1]//2) + np.argmax(axis_Histogram[img.shape[1]//2:]) 

    return left_lane, right_lane


In [8]:
def sliding_window(img, imgHeight, left_lane, right_lane, window_height, window_width):
    
    # Arrays to hold the lane index
    right_lane_arr = []
    left_lane_arr = []

    left_y = []
    right_y = []
    
    # Creating the window on the lanes bases
    num_of_windows = imgHeight//window_height

    # Move the window on the lane
    for i in range(num_of_windows):
        left_lane_win = img[imgHeight - window_height:imgHeight, left_lane - window_width/2:left_lane + window_width/2]
        right_lane_win = img[imgHeight - window_height:imgHeight, right_lane - window_width/2:right_lane + window_width/2]

        # get the white pixels in the left and right window
        left_non_zero = np.argwhere(left_lane_win = 1)
        right_non_zero = np.argwhere(right_lane_win = 1)

        # add the index to the lane arrays
        left_new_index = left_non_zero[:,1] + (left_lane - window_width//2)
        right_new_index = right_non_zero[:,1] + (right_lane - window_width//2)

        left_lane_arr = np.append(left_new_index)
        right_lane_arr = np.append(right_new_index)

        # Lanes Y index
        left_y[i],right_y[i] = imgHeight - i*window_height

        # update the window base
        left_lane = np.mean(left_new_index)
        right_lane = np.mean(right_new_index)
    # meter to pixel 
    meter_pixel = 30 / 720     # Assume 30 meters in 720 pixels

    # Curve fitting(2nd degree poynomial) for the points from the window
    left_curve = np.polyfit(left_y*meter_pixel, left_lane_arr, 2)
    right_curve = np.polyfit(right_y*meter_pixel, right_lane_arr, 2)

    return left_curve, right_curve


In [ ]:
def get_Curvature(left_curve, right_curve, y_value, meter_pixel = 30/720):

    # polynomial fitting = Ax^2 + Bx + C
    A_left, B_left, C_left = left_curve
    A_right, B_right, C_right = right_curve

    # If A = -ve , then the direction of curvature is at the left & vice-versa
    if A_left and A_right > 0:
        direction = 'right'
    elif A_left and A_right < 0:
        direction = 'left'
    else:
        direction = 'straight'
    # Radius of curvature = ((1 + (2Ax + B)**2)**(3/2)) / |2A|
    L_curvature = ((1 + (2 * A_left * y_value * meter_pixel + B_left)**2)**1.5) / np.abs(2 * A_left)
    R_curvature = ((1 + (2 * A_right * y_value * meter_pixel + B_right)**2)**1.5) / np.abs(2 * A_right)

    return direction, L_curvature, R_curvature
    


    